In [5]:
algo = "gru4rec_Reminder"
budget = 100
windows = [1,2,3,4,5]
window = windows[0]
cmd_list = [["python", "conf/in/" + str(algo) + "-config_" + str(i) + "-window_" + str(window) + ".yml"] for i in range(budget)]

In [6]:
import theano
from theano import tensor as T

In [2]:
x = theano.tensor.ivector()

In [8]:
r = T.ivector()
new_r = T.set_subtensor(r[10:], 5)

In [9]:
new_r

IncSubtensor{Set;int64::}.0

In [1]:
from IPython.core.debugger import set_trace



In [7]:
def add_to_life_universe_everything(x):
    answer = 42
    
    answer += x
    set_trace()
    return answer

# add_to_life_universe_everything(12)

In [9]:
add_to_life_universe_everything(12)

> <ipython-input-7-ec0c16a426ff>(6)add_to_life_universe_everything()
      4     answer += x
      5     set_trace()
----> 6     return answer
      7 
      8 # add_to_life_universe_everything(12)

ipdb> x
12
ipdb> continue


54

In [11]:
from pathlib import Path
if True:
    path1 = Path(r'path/to/file.py')

In [7]:
import importlib
from pathlib import Path
import sys
import time
import os
import glob
import traceback
import socket
import numpy as np
import pandas as pd
import yaml
from _datetime import timezone, datetime
import pickle

In [2]:
vocab = '../../bert4rec_3/data/ml-1mSimon.vocab'

In [6]:
with open('../../bert4rec_3/data/ml-1mSimon.vocab', 'rb') as f:
    x = pickle.load(f)

ModuleNotFoundError: No module named 'vocab'

In [10]:
stream = open('../../session-aware_rc_2020/code/conf/retailrocket_sa.yml')
conf = yaml.load(stream)

<ipython-input-10-ea0f4d7dd9bd>:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  conf = yaml.load(stream)


In [27]:
def load_class(path):
    '''
    Load a class from the path in the configuration
        --------
        path : dict of dicts
            Path to the class, e.g., algorithms.knn.cknn.ContextKNNN
    '''
    module_name, class_name = path.rsplit('.', 1)

    Class = getattr(importlib.import_module(module_name), class_name)
    return Class

In [29]:
path = 'algorithm.'+'baselines.sr.SequentialRules'
module_name, class_name = path.rsplit('.', 1)

In [35]:
def create_algorithms_dict(list):
    '''
    Create algorithm instances from the list of algorithms in the configuration
        --------
        list : list of dicts
            Dicts represent a single algorithm with class, a key, and optionally a param dict
    '''

    algorithms = {}
    for algorithm in list:
        Class = SequentialRules

        default_params = algorithm['params'] if 'params' in algorithm else {}
        random_params = {}
        params = {**default_params, **random_params}
        del default_params, random_params

#         if 'params' in algorithm:
#             if 'algorithms' in algorithm['params']:
#                 hybrid_algorithms = create_algorithms_dict(algorithm['params']['algorithms'])
#                 params['algorithms'] = []
#                 a_keys = []
#                 for k, a in hybrid_algorithms.items():
#                     params['algorithms'].append(a)
#                     a_keys.append(k)

        # instance = Class( **params )
        key = algorithm['key'] if 'key' in algorithm else algorithm['class']
        if 'params' in algorithm:
            if 'algorithms' in algorithm['params']:
                for k, val in params.items():
                    if k == 'algorithms':
                        for pKey in a_keys:
                            key += '-' + pKey
                    elif k == 'file':
                        key += ''
                    else:
                        key += '-' + str(k) + "=" + str(val)
                        key = key.replace(',', '_')

            else:
                for k, val in params.items():
                    if k != 'file':
                        key += '-' + str(k) + "=" + str(val)
                        key = key.replace(',', '_')
                    # key += '-' + '-'.join( map( lambda x: str(x[0])+'='+str(x[1]), params.items() ) )

        if 'params_var' in algorithm:
            for k, var in algorithm['params_var'].items():
                for val in var:
                    params[k] = val  # params.update({k: val})
                    kv = k
                    for v in val:
                        kv += '-' + str(v)
                    instance = Class(**params)
                    algorithms[key + kv] = instance
        else:
            instance = Class(**params)
            algorithms[key] = instance

    return algorithms

In [48]:
a = [conf['algorithms'][0]]
a

[{'class': 'baselines.sr.SequentialRules',
  'params': {'steps': 20, 'weighting': 'quadratic'},
  'key': 'sr'}]

In [50]:
b = create_algorithms_dict(a)

In [51]:
b.items()

dict_items([('sr-steps=20-weighting=quadratic', <__main__.SequentialRules object at 0x0000027612E1F730>)])

In [1]:
def eval_algorithm(train, test, key, algorithm, eval, metrics, results, conf, slice=None, iteration=None, out=True):
    '''
    Evaluate one single algorithm
        --------
        train : Dataframe
            Training data
        test: Dataframe
            Test set
        key: string
            The automatically created key string for the algorithm
        algorithm: algorithm object
            Just the algorithm object, e.g., ContextKNN
        eval: module
            The module for evaluation, e.g., evaluation.evaluation_last
        metrics: list of Metric
            Optional string to add to the file name
        results: dict
            Result dictionary
        conf: dict
            Configuration dictionary
        slice: int
            Optional index for the window slice
    '''
    ts = time.time()
    print('fit ', key)
    # send_message( 'training algorithm ' + key )

    if hasattr(algorithm, 'init'):
        algorithm.init(train, test, slice=slice)

    for m in metrics:
        if hasattr(m, 'start'):
            m.start(algorithm)

    algorithm.fit(train, test)
    print(key, ' time: ', (time.time() - ts))

    if 'results' in conf and 'pickle_models' in conf['results']:
        try:
            save_model(key, algorithm, conf)
        except Exception:
            print('could not save model for ' + key)

    for m in metrics:
        if hasattr(m, 'start'):
            m.stop(algorithm)

    results[key] = eval.evaluate_sessions(algorithm, metrics, test, train)
    if out:
        write_results_csv({key: results[key]}, conf, extra=key, iteration=iteration)

In [ ]:
algorithms = create_algorithms_dict(conf['algorithms'])
    metrics = create_metric_list(conf['metrics'])
    evaluation = load_evaluation(conf['evaluation'])

    buys = pd.DataFrame()

    if 'type' in conf['data']:
        if conf['data']['type'] == 'hdf':  # hdf5 file
            if 'opts' in conf['data']:
                # ( path, file, sessions_train=None, sessions_test=None, slice_num=None, train_eval=False )
                train, test = dl.load_data_session_hdf(conf['data']['folder'], conf['data']['prefix'], slice_num=slice,
                                                       **conf['data']['opts'])
            else:
                train, test = dl.load_data_session_hdf(conf['data']['folder'], conf['data']['prefix'], slice_num=slice)
        # elif conf['data']['type'] == 'csv': # csv file
    else:  # csv file (default)
        if 'opts' in conf['data']:
            train, test = dl.load_data_session(conf['data']['folder'], conf['data']['prefix'], slice_num=slice,
                                               **conf['data']['opts'])
        else:
            train, test = dl.load_data_session(conf['data']['folder'], conf['data']['prefix'], slice_num=slice)
        if 'buys' in conf['data'] and 'file_buys' in conf['data']:
            buys = dl.load_buys(conf['data']['folder'], conf['data']['file_buys'])  # load buy actions in addition
    # else:
    #     raise RuntimeError('Unknown data type: {}'.format(conf['data']['type']))

    for m in metrics:
        m.init(train)
        if hasattr(m, 'set_buys'):
            m.set_buys(buys, test)

    results = {}

In [ ]:
for k, a in algorithms.items():
        eval_algorithm(train, test, k, a, evaluation, metrics, results, conf, slice=slice, iteration=slice)

In [14]:
def check_data(train, test):
    if 'ItemId' in train.columns and 'SessionId' in train.columns:

        new_in_test = set(test.ItemId.unique()) - set(train.ItemId.unique())
        if len(new_in_test) > 0:
            print('WAAAAAARRRNIIIIING: new items in test set')

        session_min_train = train.groupby('SessionId').size().min()
        if session_min_train == 0:
            print('WAAAAAARRRNIIIIING: session length 1 in train set')

        session_min_test = test.groupby('SessionId').size().min()
        if session_min_test == 0:
            print('WAAAAAARRRNIIIIING: session length 1 in train set')

        sess_train = train.SessionId.unique()
        sess_test = test.SessionId.unique()

        if not all(sess_train[i] <= sess_train[i + 1] for i in range(len(sess_train) - 1)):
            print('WAAAAAARRRNIIIIING: train sessions not sorted by id')
            train.sort_values(['SessionId', 'Time'], inplace=True)
            print(' -- corrected the order')

        if not all(sess_test[i] <= sess_test[i + 1] for i in range(len(sess_test) - 1)):
            print('WAAAAAARRRNIIIIING: test sessions not sorted by id')
            test.sort_values(['SessionId', 'Time'], inplace=True)
            print(' -- corrected the order')

        test.SessionId.unique()

    else:
        print('data check not possible due to individual column names')

In [12]:
def rename_cols(df): #TODO: handle this part daynamicly
    names = {}
    names['item_id'] = 'ItemId'
    names['sessionId'] = 'SessionId'
    names['user_id'] = 'UserId'
    names['created_at'] = 'Time'

    names['itemId'] = 'ItemId'
    names['session_id'] = 'SessionId'
    names['userId'] = 'UserId'
    names['eventdate'] = 'Time'

    names['itemid'] = 'ItemId'
    names['session_id'] = 'SessionId'
    names['visitorid'] = 'UserId'
    names['timestamp'] = 'Time'

    for col in list(df.columns):
        if col in names:
            df[names[col]] = df[col]
            del df[col]

    return df

In [11]:
def prepare_data_session(train, test, sessions_train=None, sessions_test=None):
    train = rename_cols(train)
    test = rename_cols(test)

    if (sessions_train != None):
        keep = train.sort_values('Time', ascending=False).SessionId.unique()[:(sessions_train - 1)]
        train = train[np.in1d(train.SessionId, keep)]
        test = test[np.in1d(test.ItemId, train.ItemId)]

    if (sessions_test != None):
        keep = test.SessionId.unique()[:(sessions_test - 1)]
        test = test[np.in1d(test.SessionId, keep)]

    session_lengths = test.groupby('SessionId').size()
    test = test[np.in1d(test.SessionId, session_lengths[session_lengths > 1].index)]

    # output
    data_start = datetime.fromtimestamp(train.Time.min(), timezone.utc)
    data_end = datetime.fromtimestamp(train.Time.max(), timezone.utc)

    print('Loaded train set\n\tEvents: {}\n\tSessions: {}\n\tItems: {}\n\tSpan: {} / {}\n'.
          format(len(train), train.SessionId.nunique(), train.ItemId.nunique(), data_start.date().isoformat(),
                 data_end.date().isoformat()))

    data_start = datetime.fromtimestamp(test.Time.min(), timezone.utc)
    data_end = datetime.fromtimestamp(test.Time.max(), timezone.utc)

    print('Loaded test set\n\tEvents: {}\n\tSessions: {}\n\tItems: {}\n\tSpan: {} / {}\n'.
          format(len(test), test.SessionId.nunique(), test.ItemId.nunique(), data_start.date().isoformat(),
                 data_end.date().isoformat()))

    check_data(train, test)

    return (train, test)

In [9]:
def load_data_session_hdf(path, file, sessions_train=None, sessions_test=None, slice_num=None,
                          train_eval=False):
    '''
       [HDF5 format] Loads a tuple of training and test set with the given parameters.

       Parameters
       --------
       path : string
           Base path to look in for the prepared data files
       file : string
           Prefix of  the dataset you want to use.
           "yoochoose-clicks-full" loads yoochoose-clicks-full_train_full.txt and yoochoose-clicks-full_test.txt
       rows_train : int or None
           Number of rows to load from the training set file.
           This option will automatically filter the test set to only retain items included in the training set.
       rows_test : int or None
           Number of rows to load from the test set file.
       slice_num :
           Adds a slice index to the constructed file_path
           yoochoose-clicks-full_train_full.0.txt
       density : float
           Percentage of the sessions to randomly retain from the original data (0-1).
           The result is cached for the execution of multiple experiments.
       Returns
       --------
       out : tuple of pandas.DataFrame
           (train, test)

       '''

    print('START load data')

    split = ''
    if (slice_num != None and isinstance(slice_num, int)):
        split = '.' + str(slice_num)

    # train_appendix = '_train_full'
    # test_appendix = '_test'
    train_key = 'train'
    test_key = 'test'
    if train_eval:
        # train_appendix = '_train_tr'
        # test_appendix = '_train_valid'
        train_key = 'valid_train'
        test_key = 'valid_test'

    # train = pd.read_csv(path + file + train_appendix + split + '.txt', sep='\t', dtype={'ItemId': np.int64})
    # test = pd.read_csv(path + file + test_appendix + split + '.txt', sep='\t', dtype={'ItemId': np.int64})

    sessions_path = os.path.join(path, file + split + '.hdf')
    train = pd.read_hdf(sessions_path, train_key)
    test = pd.read_hdf(sessions_path, test_key)

    train, test = prepare_data_session(train, test, sessions_train,
                                       sessions_test)  # (train, test, sessions_train=None, sessions_test=None)

    print('!!!!!!!!! File: ' + file + split + '.hdf')

    return (train, test)

In [15]:
a = load_data_session_hdf(path="../../session-aware_rc_2020/code/data/retailrocket/prepared/", file="events")

START load data
Loaded train set
	Events: 4311703
	Sessions: 843986
	Items: 1739
	Span: 2017-10-29 / 2018-01-22

Loaded test set
	Events: 1570
	Sessions: 310
	Items: 131
	Span: 2017-11-06 / 2018-01-22

!!!!!!!!! File: events.hdf


In [20]:
a[1]

,UserId,Time,SessionId,ItemId
9537,1,1.514119e+09,1553,1392
9538,1,1.514119e+09,1553,1393
9539,1,1.514119e+09,1553,1169
9540,1,1.514119e+09,1553,1539
9541,1,1.514119e+09,1553,1564
...,...,...,...,...
4306580,309,1.513229e+09,842782,1392
4306581,309,1.513229e+09,842782,1181
4306582,309,1.513229e+09,842782,1389
4314828,310,1.515953e+09,844296,1392


In [21]:
def evaluate_sessions(pr, metrics, test_data, train_data, items=None, session_key='SessionId',
                                 user_key='UserId', item_key='ItemId', time_key='Time'):
    """
    Evaluates the HGRU4Rec network wrt. recommendation accuracy measured by recall@N and MRR@N.
    Concatenates train sessions to test sessions to bootstrap the hidden states of the HGRU.
    The number of the last sessions of each user that are used in the bootstrapping is controlled by `bootstrap_length`.

    Parameters
    --------
    pr : gru4rec.HGRU4Rec
        A trained instance of the HGRU4Rec network.
    train_data : pandas.DataFrame
        Train data. It contains the transactions of the test set. It has one column for session IDs,
        one for item IDs and one for the timestamp of the events (unix timestamps).
        It must have a header. Column names are arbitrary, but must correspond to the keys you use in this function.
    test_data : pandas.DataFrame
        Test data. Same format of train_data.
    items : 1D list or None
        The list of item ID that you want to compare the score of the relevant item to.
        If None, all items of the training set are used. Default value is None.
    cut_off : int
        Cut-off value (i.e. the length of the recommendation list; N for recall@N and MRR@N). Default value is 20.
    batch_size : int
        Number of events bundled into a batch during evaluation. Speeds up evaluation.
         If it is set high, the memory consumption increases. Default value is 100.
    break_ties : boolean
        Whether to add a small random number to each prediction value in order to break up possible ties,
        which can mess up the evaluation.
        Defaults to False, because (1) GRU4Rec usually does not produce ties, except when the output saturates;
        (2) it slows down the evaluation.
        Set to True is you expect lots of ties.
    output_rankings: boolean
        If True, stores the predicted ranks of every event in test data into a Pandas DataFrame
        that is returned by this function together with the metrics.
        Notice that predictors models do not provide predictions for the first event in each session. (default: False)
    bootstrap_length: int
        Number of sessions in train data used to bootstrap the hidden state of the predictor,
        starting from the last training session of each user.
        If -1, consider all sessions. (default: -1)
    session_key : string
        Header of the session ID column in the input file (default: 'SessionId')
    user_key : string
        Header of the user ID column in the input file (default: 'UserId')
    item_key : string
        Header of the item ID column in the input file (default: 'ItemId')
    time_key : string
        Header of the timestamp column in the input file (default: 'Time')

    Returns
    --------
    out : tuple
        (Recall@N, MRR@N[, DataFrame with the detailed predicted ranks])

    """

    actions = len(test_data)
    sessions = len(test_data[session_key].unique())
    count = 0
    print('START evaluation of ', actions, ' actions in ', sessions, ' sessions')

    sc = time.clock();
    st = time.time();

    time_sum = 0
    time_sum_clock = 0
    time_count = 0

    for m in metrics:
        m.reset();

    # In case someone would try to run with both items=None and not None on the same model
    # without realizing that the predict function needs to be replaced
    # pr.predict = None

    items_to_predict = train_data[item_key].unique()

    # use the training sessions of the users in test_data to bootstrap the state of the user RNN
    test_users = test_data[user_key].unique()
    train_data = train_data[train_data[user_key].isin(test_users)].copy()

    # concatenate training and test sessions
    train_data['in_eval'] = False
    test_data['in_eval'] = True
    if pr.support_users(): # e.g. hgru4rec
        if pr.predict_with_training_data():
            test_data = pd.concat([train_data, test_data])

    test_data.sort_values([user_key, session_key, time_key], inplace=True)
    test_data = test_data.reset_index(drop=True)

    offset_sessions = np.zeros(test_data[session_key].nunique() + 1, dtype=np.int32)
    length_session = np.zeros(test_data[session_key].nunique(), dtype=np.int32)
    offset_sessions[1:] = test_data.groupby([user_key, session_key]).size().cumsum() # offset_sessions[1:] = test_data.groupby(session_key).size().cumsum()
    length_session[0:] = test_data.groupby([user_key, session_key]).size() # length_session[0:] = test_data.groupby(session_key).size()

    current_session_idx = 0
    # pos: to iterate over test data to retrieve the current session and it's first interaction
    pos = offset_sessions[current_session_idx] # index of the first element of the current session in the test data
    position = 0  # position (index) of the current element in the current session
    finished = False

    prev_sid = -1
    while not finished:

        if count % 1000 == 0:
            print('    eval process: ', count, ' of ', len(test_data), ' actions: ', (count / len(test_data) * 100.0), ' % in',
                  (time.time() - st), 's')


        crs = time.clock();
        trs = time.time();

        current_item = test_data[item_key][pos]
        current_session = test_data[session_key][pos]
        current_user = test_data[user_key][pos] # current_user = test_data[user_key][pos] if user_key is not None else -1
        ts = test_data[time_key][pos]
        rest = test_data[item_key][
               pos + 1:offset_sessions[current_session_idx] + length_session[current_session_idx]].values

        if prev_sid != current_session:
            prev_sid = current_session
            if hasattr(pr, 'predict_for_extended_model'):
                past_items = pr.predict_for_extended_model(current_user)
                for past_item in past_items:
                    pr.predict_next(current_session, past_item, current_user, items_to_predict)  # to update the state for the current session, we do not need the predictions

        if test_data['in_eval'][pos] == True:
            for m in metrics:
                if hasattr(m, 'start_predict'):
                    m.start_predict(pr)

        if pr.support_users():  # session-aware (e.g. hgru4rec)
            preds = pr.predict_next(current_session, current_item, current_user, items_to_predict, timestamp=ts)
        else:  # session-based (e.g. sknn)
            preds = pr.predict_next(current_session, current_item, items_to_predict, timestamp=ts)  # without user_id

        if test_data['in_eval'][pos] == True:
            for m in metrics:
                if hasattr(m, 'stop_predict'):
                    m.stop_predict(pr)

        preds[np.isnan(preds)] = 0
         #  preds += 1e-8 * np.random.rand(len(preds)) #Breaking up ties
        preds.sort_values(ascending=False, inplace=True)

        time_sum_clock += time.clock() - crs
        time_sum += time.time() - trs
        time_count += 1

        count += 1

        if test_data['in_eval'][pos] == True:
            for m in metrics:
                if hasattr(m, 'add_multiple'):
                    m.add_multiple(preds, rest, for_item=current_item, session=current_session, position=position)
                elif hasattr(m, 'add'):
                    m.add(preds, rest[0], for_item=current_item, session=current_session, position=position)

        pos += 1
        position += 1

        # check if we make prediction for all items of the current session (except the last one)
        if pos + 1 == offset_sessions[current_session_idx] + length_session[current_session_idx]:
            current_session_idx += 1 # start the next session

            if current_session_idx == test_data[session_key].nunique(): # if we check all sessions of the test data
                finished = True # finish the evaluation

            # retrieve the index of the first interaction of the next session we want to iterate over
            pos = offset_sessions[current_session_idx]
            position = 0 # reset the first position of the first interaction in the session
            # increment count because of the last item of the session (which we do not make prediction for)
            count += 1


    print('END evaluation in ', (time.clock() - sc), 'c / ', (time.time() - st), 's')
    print('    avg rt ', (time_sum / time_count), 's / ', (time_sum_clock / time_count), 'c')
    print('    time count ', (time_count), 'count/', (time_sum), ' sum')

    res = []
    for m in metrics:
        if type(m).__name__ == 'Time_usage_testing':
            res.append(m.result_second(time_sum_clock / time_count))
            res.append(m.result_cpu(time_sum_clock / time_count))
        else:
            res.append(m.result())

    return res

In [16]:
test_data = pd.read_hdf('../../session-aware_RC_2020/events.hdf', 'test')
train_data = pd.read_hdf('../../session-aware_RC_2020/events.hdf', 'train')

In [17]:
session_key = 'session_id'
user_key = 'visitorid'
item_key = 'itemid'
time_key = 'timestamp'

In [18]:
items_to_predict = train_data[item_key].unique()
items_to_predict.shape

(39974,)

In [ ]:
actions = len(test_data)
sessions = len(test_data[session_key].unique())
count = 0
print('START evaluation of ', actions, ' actions in ', sessions, ' sessions')


time_sum = 0
time_sum_clock = 0
time_count = 0

for m in metrics:
    m.reset();

# In case someone would try to run with both items=None and not None on the same model
# without realizing that the predict function needs to be replaced
# pr.predict = None

items_to_predict = train_data[item_key].unique()

# use the training sessions of the users in test_data to bootstrap the state of the user RNN
test_users = test_data[user_key].unique()
train_data = train_data[train_data[user_key].isin(test_users)].copy()

# concatenate training and test sessions
train_data['in_eval'] = False
test_data['in_eval'] = True
if pr.support_users(): # e.g. hgru4rec
    if pr.predict_with_training_data():
        test_data = pd.concat([train_data, test_data])

test_data.sort_values([user_key, session_key, time_key], inplace=True)
test_data = test_data.reset_index(drop=True)

offset_sessions = np.zeros(test_data[session_key].nunique() + 1, dtype=np.int32)
length_session = np.zeros(test_data[session_key].nunique(), dtype=np.int32)
offset_sessions[1:] = test_data.groupby([user_key, session_key]).size().cumsum() # offset_sessions[1:] = test_data.groupby(session_key).size().cumsum()
length_session[0:] = test_data.groupby([user_key, session_key]).size() # length_session[0:] = test_data.groupby(session_key).size()

current_session_idx = 0
# pos: to iterate over test data to retrieve the current session and it's first interaction
pos = offset_sessions[current_session_idx] # index of the first element of the current session in the test data
position = 0  # position (index) of the current element in the current session
finished = False

prev_sid = -1
while not finished:

    if count % 1000 == 0:
        print('    eval process: ', count, ' of ', len(test_data), ' actions: ', (count / len(test_data) * 100.0), ' % in',
              (time.time() - st), 's')


    crs = time.clock();
    trs = time.time();

    current_item = test_data[item_key][pos]
    current_session = test_data[session_key][pos]
    current_user = test_data[user_key][pos] # current_user = test_data[user_key][pos] if user_key is not None else -1
    ts = test_data[time_key][pos]
    rest = test_data[item_key][
           pos + 1:offset_sessions[current_session_idx] + length_session[current_session_idx]].values

    if prev_sid != current_session:
        prev_sid = current_session
        if hasattr(pr, 'predict_for_extended_model'):
            past_items = pr.predict_for_extended_model(current_user)
            for past_item in past_items:
                pr.predict_next(current_session, past_item, current_user, items_to_predict)  # to update the state for the current session, we do not need the predictions

    if test_data['in_eval'][pos] == True:
        for m in metrics:
            if hasattr(m, 'start_predict'):
                m.start_predict(pr)

    if pr.support_users():  # session-aware (e.g. hgru4rec)
        preds = pr.predict_next(current_session, current_item, current_user, items_to_predict, timestamp=ts)
    else:  # session-based (e.g. sknn)
        preds = pr.predict_next(current_session, current_item, items_to_predict, timestamp=ts)  # without user_id

    if test_data['in_eval'][pos] == True:
        for m in metrics:
            if hasattr(m, 'stop_predict'):
                m.stop_predict(pr)

    preds[np.isnan(preds)] = 0
     #  preds += 1e-8 * np.random.rand(len(preds)) #Breaking up ties
    preds.sort_values(ascending=False, inplace=True)

    time_sum_clock += time.clock() - crs
    time_sum += time.time() - trs
    time_count += 1

    count += 1

    if test_data['in_eval'][pos] == True:
        for m in metrics:
            if hasattr(m, 'add_multiple'):
                m.add_multiple(preds, rest, for_item=current_item, session=current_session, position=position)
            elif hasattr(m, 'add'):
                m.add(preds, rest[0], for_item=current_item, session=current_session, position=position)

    pos += 1
    position += 1

    # check if we make prediction for all items of the current session (except the last one)
    if pos + 1 == offset_sessions[current_session_idx] + length_session[current_session_idx]:
        current_session_idx += 1 # start the next session

        if current_session_idx == test_data[session_key].nunique(): # if we check all sessions of the test data
            finished = True # finish the evaluation

        # retrieve the index of the first interaction of the next session we want to iterate over
        pos = offset_sessions[current_session_idx]
        position = 0 # reset the first position of the first interaction in the session
        # increment count because of the last item of the session (which we do not make prediction for)
        count += 1


print('END evaluation in ', (time.clock() - sc), 'c / ', (time.time() - st), 's')
print('    avg rt ', (time_sum / time_count), 's / ', (time_sum_clock / time_count), 'c')
print('    time count ', (time_count), 'count/', (time_sum), ' sum')

res = []
for m in metrics:
    if type(m).__name__ == 'Time_usage_testing':
        res.append(m.result_second(time_sum_clock / time_count))
        res.append(m.result_cpu(time_sum_clock / time_count))
    else:
        res.append(m.result())

In [14]:
offset_sessions[1]

8

In [20]:
offset_sessions = np.zeros(test_data[session_key].nunique() + 1, dtype=np.int32)
length_session = np.zeros(test_data[session_key].nunique(), dtype=np.int32)
offset_sessions[1:] = test_data.groupby([user_key, session_key]).size().cumsum() # offset_sessions[1:] = test_data.groupby(session_key).size().cumsum()
length_session[0:] = test_data.groupby([user_key, session_key]).size() # length_session[0:] = test_data.groupby(session_key).size()

In [22]:
current_session_idx = 0
# pos: to iterate over test data to retrieve the current session and it's first interaction
pos = offset_sessions[current_session_idx] # index of the first element of the current session in the test data
position = 0  # position (index) of the current element in the current session
finished = False

In [25]:
current_session = test_data[session_key][pos]

In [26]:
current_session

111